In [1]:
import pandas as pd

An aisle is a long narrow gap that people can walk along between rows of seats in a public building such as a church or between rows of shelves in a supermarket.

In [8]:
DATA_PATH = "../../embedding/data"

In [14]:
import numpy as np 
import pandas as pd

import os 

CSV_PATHS = []
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        CSV_PATHS.append(path)

In [22]:
for path in CSV_PATHS:
    print(path)
    display(pd.read_csv(path).head())

../../embedding/data/products.csv


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


../../embedding/data/order_products__prior.csv


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


../../embedding/data/orders.csv


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


../../embedding/data/order_products__train.csv


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


../../embedding/data/sample_submission.csv


,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


**What we need is Order id and its corresponding products bought**

- To obtain this, we have order_id and product_id from **order_products__train.csv**
- Map product_id to product_name from **products.csv**

In [24]:
products_df = pd.read_csv("../../embedding/data/products.csv")
display(products_df.head())

orders_df = pd.read_csv("../../embedding/data/order_products__train.csv")
display(orders_df.head())

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [26]:
print(products_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB
None


In [27]:
print(orders_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB
None


**There are not any null values in our dataframes**

In [29]:
transactions = pd.merge(orders_df, products_df, on="product_id", how="left")
transactions.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16
2,1,10246,3,0,Organic Celery Hearts,83,4
3,1,49683,4,0,Cucumber Kirby,83,4
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15


In [30]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   order_id           1384617 non-null  int64 
 1   product_id         1384617 non-null  int64 
 2   add_to_cart_order  1384617 non-null  int64 
 3   reordered          1384617 non-null  int64 
 4   product_name       1384617 non-null  object
 5   aisle_id           1384617 non-null  int64 
 6   department_id      1384617 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 73.9+ MB


In [35]:
transactions.groupby('order_id')["product_name"].unique()[:2]

order_id
1     [Bulgarian Yogurt, Organic 4% Milk Fat Whole M...
36    [Grated Pecorino Romano Cheese, Spring Water, ...
Name: product_name, dtype: object

In [37]:
order_transactions = pd.DataFrame(transactions.groupby('order_id')['product_name'].nunique().index)
order_transactions.head()

,order_id
0,1
1,36
2,38
3,96
4,98


In [38]:
order_transactions["products"] = transactions.groupby('order_id')["product_name"].unique().values
order_transactions.head()

,order_id,products
0,1,"[Bulgarian Yogurt, Organic 4% Milk Fat Whole M..."
1,36,"[Grated Pecorino Romano Cheese, Spring Water, ..."
2,38,"[Shelled Pistachios, Organic Biologique Limes,..."
3,96,"[Roasted Turkey, Organic Cucumber, Organic Gra..."
4,98,"[Natural Spring Water, Organic Orange Juice Wi..."


In [39]:
order_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131209 entries, 0 to 131208
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   order_id  131209 non-null  int64 
 1   products  131209 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.0+ MB


In [40]:
from apyori import apriori

In [43]:
transactions_apriori = order_transactions["products"].to_list()
rules = apriori(transactions=transactions_apriori, min_support = 0.00030, min_confidence=0.01, min_lift=3, min_length=2, max_length=2)
results = list(rules)



In [44]:
def inspect(results):
    '''
    function to put the result in well organised pandas dataframe
    '''
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Item #1', 'Item #2', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame.head()

,Item #1,Item #2,Support,Confidence,Lift
0,0% Greek Strained Yogurt,Clementines,0.000602,0.191283,18.577416
1,0% Greek Strained Yogurt,Hass Avocados,0.000343,0.108959,6.281362
2,0% Greek Strained Yogurt,Raspberries,0.000312,0.099274,3.972428
3,0% Greek Strained Yogurt,Soda,0.000450,0.142857,12.438051
4,0% Greek Strained Yogurt,Trail Mix,0.000389,0.123487,30.398807


In [46]:
resultsinDataFrame.nlargest(n=20, columns='Lift')

,Item #1,Item #2,Support,Confidence,Lift
2880,Unsweetened Whole Milk Peach Greek Yogurt,Unsweetened Whole Milk Strawberry Yogurt,0.000381,0.531915,606.887142
2878,Unsweetened Whole Milk Blueberry Greek Yogurt,Unsweetened Whole Milk Strawberry Yogurt,0.000328,0.500000,570.473913
2879,Unsweetened Whole Milk Mixed Berry Greek Yogurt,Unsweetened Whole Milk Strawberry Yogurt,0.000312,0.493976,563.600733
2877,Unsweetened Blackberry Water,Unsweetened Watermelon Water,0.000305,0.449438,440.077142
1300,Organic 4 Months Butternut Squash Carrots Appl...,Stage 1 Apples Sweet Potatoes Pumpkin & Bluebe...,0.000335,0.453608,407.654004
1538,Organic Blended Raspberry Whole Milk Greek Yogurt,Organic Greek Whole Milk Blended Strawberry Yo...,0.000305,0.416667,364.469444
57,Almond Milk Blueberry Yogurt,Almond Milk Peach Yogurt,0.000594,0.430939,353.394406
2433,"Organic Pears, Peas and Broccoli Puree Stage 1",Stage 1 Apples Sweet Potatoes Pumpkin & Bluebe...,0.000320,0.365217,328.217868
37,Acai Berry Chia Bar,Coconut Chia Bar,0.000381,0.409836,316.318708
58,Almond Milk Blueberry Yogurt,Almond Milk Strawberry Yogurt,0.000610,0.441989,310.122611


**We can observe that with different kinds of greek yogurt, Unsweetened Whole Milk Strawberry Yogurt is also purchased. Hence the store should add deals with greek yogurt, and blueberry yogurt**